# General configuration - object detection
**Object Detection API configuration**: In this step, the model for object detection is downloaded, also some copying and deletion of references are done to leave the whole scheme configured.

In [0]:
!git clone https://github.com/tensorflow/models.git
!apt-get -qq install libprotobuf-java protobuf-compiler
!protoc ./models/research/object_detection/protos/string_int_label_map.proto --python_out=.
!cp -R models/research/object_detection/ object_detection/
!rm -rf models

# Imports 

**Imports** needed to run the Object Detection API demonstration

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from scipy.stats import itemfreq

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

**Configuration** of the model to use, path to the frozen inference graph and extra config elements for the Object detection API implementation.

In [0]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('object_detection/data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
    
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Upload tie image

In [0]:
!mkdir images

In [0]:
#red tie demo
!wget http://www.disfrazjaiak.com/1294-thickbox_default/corbata-roja.jpg -O images/custom_img_1.jpg

In [0]:
!ls ./images/ #optional to validte files

In [0]:
from matplotlib.pyplot import imshow
%matplotlib inline

test_image = Image.open('./images/custom_img_1.jpg')
imshow(test_image)

In [0]:
PATH_TO_TEST_IMAGES_DIR = 'images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'custom_img_{}.jpg'.format(i)) for i in range(1, 2) ]

## Routine to get labels

In [0]:
def getLabels(boxes,classes, score):
      labels = []
      for i, box in enumerate(np.squeeze(boxes)):
        if(np.squeeze(scores)[i] > score):
          labels.append(category_index[np.squeeze(classes)[i]]['name']) 
        
      return labels

## Objects Analysis into Image

In [0]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      image_np = load_image_into_numpy_array(image)
      
      
      image_np_expanded = np.expand_dims(image_np, axis=0)
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
     
      print(image_path)
      print(getLabels(boxes, classes, 0.7))

## Optional CV [video or cam]

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

In [0]:
!mkdir videos

In [0]:
!wget https://storage.googleapis.com/videostestingobjectdetection/video_demo_baby_dog.mp4 -O videos/video_demo_baby_dog.mp4

In [0]:
out = cv2.VideoWriter('outputtestvideojapanstreet.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (960,540))

filename = 'videos/testvideojapanstreet.mp4'
cap = cv2.VideoCapture(filename)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        counter = 0
        while (True):
            ret, image_np = cap.read()
            
            counter += 1
            if ret:
              h = image_np.shape[0]
              w = image_np.shape[1]

              if counter % 1 == 0:
                image_np_expanded = np.expand_dims(image_np, axis=0)
                image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
                boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
                scores = detection_graph.get_tensor_by_name('detection_scores:0')
                classes = detection_graph.get_tensor_by_name('detection_classes:0')
                num_detections = detection_graph.get_tensor_by_name('num_detections:0')

                (boxes, scores, classes, num_detections) = sess.run(
                    [boxes, scores, classes, num_detections],
                    feed_dict={image_tensor: image_np_expanded})

                vis_util.visualize_boxes_and_labels_on_image_array(image_np,
                                                                     np.squeeze(boxes),
                                                                     np.squeeze(classes).astype(np.int32),
                                                                     np.squeeze(scores),
                                                                     category_index,
                                                                     use_normalized_coordinates=True,
                                                                     line_thickness=0,
                                                                     min_score_thresh=0.3)

              out.write(image_np)
            else:
              break
out.release()
cap.release()
cv2.destroyAllWindows()

In [0]:
!ls 

In [0]:
!pip install moviepy

In [0]:
from moviepy.editor import *

In [0]:
clip = VideoFileClip("outputtestvideojapanstreet.avi")
clip.write_videofile("outputtestvideojapanstreet.mp4", codec='libx264')

In [0]:
!ls 

In [0]:
import io
import base64
from IPython.display import HTML

In [0]:
video = io.open('videos/testvideojapanstreet.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="80%" alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

In [0]:
video = io.open('outputtestvideojapanstreet.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

## Video processing using moviepy [alternative -options]

In [0]:
def process_image(image):
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            image_process = detect_objects(image, sess, detection_graph)
            return image_process

In [0]:
def detect_objects(image_np, sess, detection_graph):
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
    return image_np

In [0]:
white_output = 'output.mp4'
clip1 = VideoFileClip("videos/video_demo_baby_dog.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!s
%time white_clip.write_videofile(white_output, audio=False)